```{contents}
```

##  Why These Algorithms Matter in Generative AI

LLMs turn text/images/audio into **embeddings** (vectors).
To retrieve relevant information during RAG, we must find the **closest vectors** to a query vector **fast**, even among millions/billions of vectors.

Exact search is too slow → **ANN (Approximate Nearest Neighbor)** is used.
The three most widely used ANN engines are:

* **FAISS** (Meta / Facebook AI)
* **HNSW** (Hierarchical Navigable Small Worlds)
* **ScaNN** (Google)

Each has different strengths and trade-offs.

---

###  **FAISS** (Facebook AI Similarity Search)

#### What it is:

FAISS is a highly optimized library from Meta for **fast similarity search** and **clustering** in large vector datasets.

#### Why it is popular:

* Very fast on **GPU**
* Very fast on CPU
* Handles **billions of vectors**
* Highly tunable indexing mechanisms
* Open-source and widely used

#### Core Algorithms in FAISS:

FAISS contains multiple index types:

##### **1. Flat Index (exact search)**

```
IndexFlatL2 / IndexFlatIP
```

* brute force
* 100% accurate
* fast only for small datasets

##### **2. IVF (Inverted File Index)**

```
IndexIVFFlat
```

* clusters vectors using k-means
* search only in few clusters
* 10×–50× faster than brute force
* small accuracy loss

##### **3. PQ (Product Quantization)**

```
IndexIVFPQ
```

* compresses embeddings
* reduces RAM by 4×–16×
* used for billion-scale datasets

##### **4. HNSW + FAISS**

FAISS can also use HNSW internally.

---

### When FAISS is best:

* GPU-accelerated search
* Large datasets (millions–billions of embeddings)
* High throughput
* Cloud + on-premise systems

---

### **HNSW** (Hierarchical Navigable Small Worlds)

#### What it is:

HNSW is a **graph-based ANN algorithm** where each vector is a node in a multi-layer graph.

#### How it works (intuition):

* Create a **multi-layer graph**
* Top layers: few nodes, long-range links
* Lower layers: dense graph, short-range links
* Search moves from top → bottom
* “Navigates” quickly toward closest neighbors

#### Why it is fast:

* logarithmic-like search time
* extremely low recall loss
* does not require GPUs
* high accuracy even with large datasets

#### Where it's used:

* **Qdrant**
* **Weaviate**
* **Milvus**
* **Pinecone** (HNSW index option)

#### Strengths:

* Super accurate
* Super fast on CPU
* Great for mid-size datasets (up to hundreds of millions)
* Works well for real-time updates

#### Weakness:

* Index building is expensive
* Memory-heavy (stores graphs = many pointers)

---

### 3. **ScaNN** (Google’s Scalable Nearest Neighbor)

#### What it is:

ScaNN (“Scalable Nearest Neighbors”) is Google’s ANN engine optimized for:

* TPU / CPU
* high recall
* high efficiency
* production search systems (YouTube, Google Search)

#### How it works:

ScaNN uses:

1. **Tree-based partitioning**
2. **Asymmetric hashing**
3. **Re-ranking** with distance-aware quantization

#### Key strengths:

* Extremely high recall
* Very fast on CPU
* Efficient memory usage
* Optimized for semantic search (text embeddings)
* Great for multi-billion–scale embeddings

#### Weakness:

* Less friendly to frequent index updates
* Harder to configure compared to FAISS/HNSW
* Python integration is weaker than FAISS

---

**Comparison Summary**

| Feature           | FAISS                     | HNSW                | ScaNN                     |
| ----------------- | ------------------------- | ------------------- | ------------------------- |
| Developed by      | Meta                      | Yury Malkov         | Google                    |
| Best Hardware     | GPU + CPU                 | CPU                 | CPU/TPU                   |
| Type              | Clustering + Quantization | Graph-based         | Tree + Compression        |
| Speed             | Very high                 | Extremely high      | Highest (CPU)             |
| Accuracy          | High                      | Very high           | Very high                 |
| Memory usage      | Medium / Low (PQ)         | High (graphs)       | Medium                    |
| Real-time updates | Moderate                  | Excellent           | Weak                      |
| Best for          | Billions of vectors       | Fast queries on CPU | Production search systems |

---

### Which One Should You Use?

#### **Use FAISS if:**

* you need GPU acceleration
* your dataset is extremely large
* you want the best performance per dollar
* you need heavy index compression

#### **Use HNSW if:**

* you're using a vector database (Qdrant, Weaviate, Milvus)
* you want high accuracy with low latency
* your dataset fits in RAM
* you need frequent inserts / deletes

#### **Use ScaNN if:**

* you're running on CPU or TPU
* you want the highest recall
* you're building a semantic search engine
* you have billions of embeddings (Google-scale workloads)

---

**Complete Intuition Summary**

| Algorithm | Intuition                                             |
| --------- | ----------------------------------------------------- |
| **FAISS** | Cluster → search few clusters instead of all vectors  |
| **HNSW**  | Build a multi-layer graph → “walk” toward the nearest |
| **ScaNN** | Partition → compress → refine                         |
